<a href="https://colab.research.google.com/github/HarikrishnanK9/Hobby_Learn/blob/main/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 34.

In [4]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [6]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from huggingface_hub import InferenceClient
import numpy as np
import os

In [7]:
def text_embedding(text) -> None:
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    return model.encode(text)

In [8]:
def generate_context(query):
    vector=text_embedding(query).tolist()

    results=collection.query(
        query_embeddings=vector,
        n_results=15,
        include=["documents"]
    )

    res = "\n".join(str(item) for item in results['documents'][0])
    return res

In [13]:
def chat_completion(system_prompt, user_prompt,length=1000):
    final_prompt=f"""<s>[INST]<<SYS>>
    {system_prompt}
    <</SYS>>

    {user_prompt} [/INST]"""
    return client.text_generation(prompt=final_prompt,max_new_tokens = length).strip()

In [14]:
df=pd.read_csv('/content/oscars.csv')
df=df.loc[df['year_ceremony'] == 2023]
df=df.dropna(subset=['film'])
df.loc[:, 'category'] = df['category'].str.lower()
df.loc[:, 'text'] = df['name'] + ' got nominated under the category, ' + df['category'] + ', for the film ' + df['film'] + ' to win the award'
df.loc[df['winner'] == False, 'text'] = df['name'] + ' got nominated under the category, ' + df['category'] + ', for the film ' + df['film'] + ' but did not win'

In [15]:
client = chromadb.Client()
collection = client.get_or_create_collection("oscars-2023")

In [16]:
docs=df["text"].tolist()
ids= [str(x) for x in df.index.tolist()]
collection.add(
    documents=docs,
    ids=ids
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 66.1MiB/s]


In [17]:
URI='http://139.84.142.100:8080'
client = InferenceClient(model=URI)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [18]:
#query="What did Ke Huy Quan work on?"
#query="Which movie won the best music award?"
query="Did Lady Gaga win an award at Oscars 2023?"
#query="Who is the music director of RRR?"
context=generate_context(query)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
context


'Monika Willi got nominated under the category, film editing, for the film Tár but did not win\nMary Zophres got nominated under the category, costume design, for the film Babylon but did not win\nMusic and Lyric by Lady Gaga and BloodPop got nominated under the category, music (original song), for the film Top Gun: Maverick but did not win\nMark Coulier, Jason Baird and Aldo Signoretti got nominated under the category, makeup and hairstyling, for the film Elvis but did not win\nSara Gunnarsdóttir and Pamela Ribon got nominated under the category, short film (animated), for the film My Year of Dicks but did not win\nDede Gardner, Jeremy Kleiner and Frances McDormand, Producers got nominated under the category, best picture, for the film Women Talking but did not win\nBaz Luhrmann, Catherine Martin, Gail Berman, Patrick McCormick and Schuyler Weiss, Producers got nominated under the category, best picture, for the film Elvis but did not win\nLaura Poitras, Howard Gertler, John Lyons, Na

In [20]:
system_prompt="""\
You are a helpful AI assistant that can answer questions on Oscar 2023 awards. Answer based on the context provided. If you cannot find the correct answerm, say I don't know. Be concise and just include the response.
"""

In [21]:
user_prompt=f"""
Based on the context:
{context}
Answer the below query:
{query}
"""

In [22]:
user_prompt


'\nBased on the context:\nMonika Willi got nominated under the category, film editing, for the film Tár but did not win\nMary Zophres got nominated under the category, costume design, for the film Babylon but did not win\nMusic and Lyric by Lady Gaga and BloodPop got nominated under the category, music (original song), for the film Top Gun: Maverick but did not win\nMark Coulier, Jason Baird and Aldo Signoretti got nominated under the category, makeup and hairstyling, for the film Elvis but did not win\nSara Gunnarsdóttir and Pamela Ribon got nominated under the category, short film (animated), for the film My Year of Dicks but did not win\nDede Gardner, Jeremy Kleiner and Frances McDormand, Producers got nominated under the category, best picture, for the film Women Talking but did not win\nBaz Luhrmann, Catherine Martin, Gail Berman, Patrick McCormick and Schuyler Weiss, Producers got nominated under the category, best picture, for the film Elvis but did not win\nLaura Poitras, Howar

In [23]:
chat_completion(system_prompt,user_prompt)


ConnectTimeout: (MaxRetryError("HTTPConnectionPool(host='139.84.142.100', port=8080): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7a7388493e50>, 'Connection to 139.84.142.100 timed out. (connect timeout=None)'))"), '(Request ID: f0776cb0-70d9-4a48-843b-a24c3ad32014)')